In [6]:
import pandas as pd
from glob import glob
from collections import *
import random
from tqdm.notebook import tqdm

In [21]:
fnames = glob('/shared/0/projects/reddit-political-affiliation/data/flair-affiliations/*.tsv')

In [26]:
fnames2 = glob('/shared/0/projects/reddit-political-affiliation/data/comment-affiliations/silver/*.tsv')

In [3]:
fnames[0]

'/shared/0/projects/reddit-political-affiliation/data/flair-affiliations/RC_2010-06.tsv'

In [4]:
!head /shared/0/projects/reddit-political-affiliation/data/flair-affiliations/RC_2010-06.tsv

FarsideSC	Conservative	Republican	Conservative	1567296214
JackFucington	Conservative	Republican	Conservative	1567296262
RedBaronsBrother	Conservative	Republican	Conservative	1567296527
WunjoRolls6	AMERICA FIRST	Republican	The_Donald	1567296528
WunjoRolls6	MAGA	Republican	The_Donald	1535762247
Znut55	MAGA	Republican	The_Donald	1567296595
TheRealGrantEdwards	AMERICA FIRST	Republican	The_Donald	1567296625
DonsGuard	Trump Supporter	Republican	AskTrumpSupporters	1567296631
DonsGuard	MAGA	Republican	The_Donald	1567736684
ButGravityAlwaysWins	Liberal	Democrat	AskALiberal	1567296644


In [22]:
random.shuffle(fnames)

In [27]:
user2aff = {}
for fname in tqdm(fnames):
    with open(fname) as f:
        for line in f:
            cols = line[:-1].split('\t')
            if len(cols) < 3:
                print(line)
                continue
            user2aff[cols[0]] = cols[2]
            
for fname in tqdm(fnames2):
    with open(fname) as f:
        for line in f:
            cols = line[:-1].split('\t')
            if len(cols) < 3:
                #print(line)
                continue
            user2aff[cols[0]] = cols[1]            

In [28]:
len(user2aff)

196774

In [34]:
max(len(x) for x in user2aff.keys())

20

In [29]:
df = defaultdict(list)
for k, v in user2aff.items():
    df['username'].append(k)
    df['label'].append(v)
df = pd.DataFrame(df)
df.head()

,username,label
0,FarsideSC,Republican
1,JackFucington,Republican
2,RedBaronsBrother,Republican
3,WunjoRolls6,Democrat
4,Znut55,Republican


In [35]:
df['binary_label'] = df.label.apply(lambda x: 1 if x == 'Democrat' else 0)

In [32]:
set(df['label'])

{'Democrat', 'Republican'}

In [36]:
df.to_csv('/shared/0/projects/reddit-political-affiliation/data/username-labels/user2label.csv', index=False)

In [38]:
import numpy as np

In [39]:
train_df, dev_df, test_df = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.8*len(df)), int(.9*len(df))])

In [42]:
len(train_df), len(dev_df), len(test_df)

(157419, 19677, 19678)

In [40]:
train_df.to_csv('/shared/0/projects/reddit-political-affiliation/data/username-labels/user2label.train.csv', index=False)
dev_df.to_csv('/shared/0/projects/reddit-political-affiliation/data/username-labels/user2label.dev.csv', index=False)
test_df.to_csv('/shared/0/projects/reddit-political-affiliation/data/username-labels/user2label.test.csv', index=False)